# Chapter 6: Dead Reckoning

## Principles of Indoor Positioning and Indoor Navigation

---

### 📚 Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand** dead reckoning principles and drift sources
2. **Implement** IMU strapdown integration (attitude, velocity, position)
3. **Apply** drift correction constraints (ZUPT, ZARU, NHC)
4. **Use** Pedestrian Dead Reckoning (PDR) with step-and-heading
5. **Process** environmental sensors (magnetometer heading, barometric altitude)
6. **Analyze** why DR alone is insufficient and requires fusion

### 📖 Book Reference

This notebook covers **Chapter 6: Dead Reckoning** with:
- **Eq. (6.2)-(6.10)**: IMU strapdown integration
- **Eq. (6.44)-(6.45)**: ZUPT zero-velocity updates
- **Eq. (6.46)-(6.50)**: Pedestrian dead reckoning (step detection, step length)
- **Eq. (6.51)-(6.54)**: Environmental sensors (magnetometer, barometer)

**Key Insight**: Dead reckoning drifts **unbounded** without external corrections!

---


## 🚀 Setup (Google Colab)

**Set the `GITHUB_REPO` variable below to your repository URL, then run the setup cell.**

Example: `GITHUB_REPO = "https://github.com/YOUR_USERNAME/IPIN_Book_Examples.git"`


In [ ]:
# ========================================
# IPIN Book Examples - Chapter 6: Dead Reckoning
# ========================================

import os
import sys

# ============ CONFIGURATION ============
GITHUB_REPO = None  # Set your repo URL, e.g., "https://github.com/username/IPIN_Book_Examples.git"
# =======================================

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    if os.path.exists('/content/IPIN_Book_Examples/core'):
        os.chdir('/content/IPIN_Book_Examples')
        print("✅ Repository already available.")
    elif GITHUB_REPO:
        print(f"📥 Cloning from {GITHUB_REPO}...")
        get_ipython().system(f'git clone {GITHUB_REPO}')
        os.chdir('/content/IPIN_Book_Examples')
        get_ipython().system('pip install -e . -q')
        print("✅ Setup from GitHub complete!")
    else:
        print("❌ ERROR: GITHUB_REPO not set!")
        print("Please set GITHUB_REPO = 'https://github.com/YOUR_USERNAME/IPIN_Book_Examples.git'")
        raise ValueError("GITHUB_REPO not configured.")
else:
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir('..')
    print(f"📂 Working directory: {os.getcwd()}")

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

get_ipython().run_line_magic('matplotlib', 'inline')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

# Import dead reckoning modules
from core.sensors import (
    strapdown_update,
    euler_to_quat,
    quat_to_euler,
    total_accel_magnitude,
    step_length,
    pdr_step_update,
    integrate_gyro_heading,
    wrap_heading,
    mag_heading,
    pressure_to_altitude,
    detect_zupt,
)

print("\n" + "="*60)
print("✅ Setup complete! Dead reckoning modules loaded.")
print("="*60)


---

# Part 1: Dead Reckoning Overview

## 1.1 What is Dead Reckoning?

**Dead Reckoning (DR)** estimates position by integrating motion measurements from **proprioceptive sensors**:

| Sensor | Measures | Platform |
|--------|----------|----------|
| **IMU** (Accelerometer + Gyroscope) | Acceleration, Angular velocity | Any |
| **Wheel Encoders** | Wheel rotation | Vehicles |
| **Pedometer** | Step count + heading | Pedestrians |

## 1.2 The Drift Problem

DR error grows **unbounded** over time due to:
- **Sensor noise**: Random walk in integrated quantities
- **Bias drift**: Slowly varying sensor offsets
- **Scale factor errors**: Imperfect calibration

| Integration Level | Error Type | Growth |
|-------------------|------------|--------|
| Attitude (1×) | Angular random walk | $\propto \sqrt{t}$ |
| Velocity (2×) | Velocity random walk | $\propto t$ |
| Position (3×) | Position random walk | $\propto t^2$ |

**Bottom line**: Pure IMU navigation is unusable after seconds to minutes!


---

# Part 2: IMU Strapdown Integration

## 2.1 Theory (Eqs. 6.2-6.10)

**Strapdown integration** propagates attitude, velocity, and position using IMU measurements:

### Attitude Update (Quaternion)
$$\mathbf{q}_{k+1} = \mathbf{q}_k \otimes \exp\left(\frac{1}{2}\boldsymbol{\omega}_k \Delta t\right)$$

### Velocity Update
$$\mathbf{v}_{k+1} = \mathbf{v}_k + \left(\mathbf{R}(\mathbf{q}_k) \mathbf{a}_k - \mathbf{g}\right) \Delta t$$

### Position Update
$$\mathbf{p}_{k+1} = \mathbf{p}_k + \mathbf{v}_k \Delta t + \frac{1}{2}\left(\mathbf{R}(\mathbf{q}_k) \mathbf{a}_k - \mathbf{g}\right) \Delta t^2$$

where:
- $\boldsymbol{\omega}_k$ = gyroscope measurement (rad/s)
- $\mathbf{a}_k$ = accelerometer measurement (m/s²)
- $\mathbf{g}$ = gravity vector ([0, 0, 9.81] m/s²)


In [ ]:
# Example 1: IMU Strapdown Integration - Demonstrating Drift
print("="*70)
print("Example 1: IMU Strapdown Integration (Drift Demonstration)")
print("="*70)

# Simulation parameters
np.random.seed(42)
dt = 0.01  # 100 Hz
duration = 30.0  # 30 seconds
N = int(duration / dt)
t = np.arange(N) * dt

# Simulate circular trajectory (true path)
omega_true = 2 * np.pi / 20  # One circle in 20 seconds
radius = 5.0
pos_true = np.zeros((N, 3))
vel_true = np.zeros((N, 3))

for k in range(N):
    theta = omega_true * t[k]
    pos_true[k, 0] = radius * np.sin(theta)  # x
    pos_true[k, 1] = radius * (1 - np.cos(theta))  # y
    vel_true[k, 0] = radius * omega_true * np.cos(theta)
    vel_true[k, 1] = radius * omega_true * np.sin(theta)

# Simulate IMU measurements with noise
gyro_noise_std = np.deg2rad(0.5)  # 0.5 deg/s
accel_noise_std = 0.05  # 0.05 m/s²
gyro_bias = np.array([0.0, 0.0, np.deg2rad(1.0)])  # 1 deg/s yaw bias

gyro_meas = np.zeros((N, 3))
accel_meas = np.zeros((N, 3))

for k in range(N):
    # True angular velocity (rotating in z)
    gyro_meas[k] = np.array([0.0, 0.0, omega_true])
    gyro_meas[k] += gyro_bias + np.random.randn(3) * gyro_noise_std
    
    # True acceleration (centripetal + gravity)
    theta = omega_true * t[k]
    a_centripetal = -omega_true**2 * radius * np.array([np.sin(theta), 1-np.cos(theta), 0])
    accel_meas[k] = a_centripetal + np.array([0, 0, 9.81])
    accel_meas[k] += np.random.randn(3) * accel_noise_std

print(f"\n📊 Simulation Parameters:")
print(f"  Duration: {duration} s at {1/dt:.0f} Hz")
print(f"  True path: Circular (radius={radius}m)")
print(f"  Gyro noise: {np.rad2deg(gyro_noise_std):.2f} deg/s")
print(f"  Gyro bias: {np.rad2deg(gyro_bias[2]):.2f} deg/s (yaw)")
print(f"  Accel noise: {accel_noise_std:.3f} m/s²")


In [ ]:
# Run strapdown integration
pos_imu = np.zeros((N, 3))
vel_imu = np.zeros((N, 3))
quat = euler_to_quat(0.0, 0.0, 0.0)  # Initial attitude

for k in range(1, N):
    # Strapdown update
    quat, vel_imu[k], pos_imu[k] = strapdown_update(
        quat, vel_imu[k-1], pos_imu[k-1],
        gyro_meas[k], accel_meas[k], dt
    )

# Compute errors
pos_error = np.linalg.norm(pos_imu[:, :2] - pos_true[:, :2], axis=1)
final_error = pos_error[-1]
total_distance = np.sum(np.linalg.norm(np.diff(pos_true[:, :2], axis=0), axis=1))

print(f"\n🚨 IMU-Only Results (NO corrections):")
print(f"  Total distance traveled: {total_distance:.1f} m")
print(f"  Final position error:    {final_error:.1f} m")
print(f"  Error as % of distance:  {100*final_error/total_distance:.1f}%")
print(f"  Max error:               {pos_error.max():.1f} m")
print(f"\n⚠️ CRITICAL: Error grows UNBOUNDED with time!")


In [ ]:
# Visualize IMU drift
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Trajectory
ax1 = axes[0]
ax1.plot(pos_true[:, 0], pos_true[:, 1], 'g-', linewidth=2, label='True Path')
ax1.plot(pos_imu[:, 0], pos_imu[:, 1], 'r--', linewidth=2, label='IMU-Only')
ax1.plot(pos_true[0, 0], pos_true[0, 1], 'go', markersize=10, label='Start')
ax1.plot(pos_true[-1, 0], pos_true[-1, 1], 'g^', markersize=10, label='End (True)')
ax1.plot(pos_imu[-1, 0], pos_imu[-1, 1], 'r^', markersize=10, label='End (IMU)')
ax1.set_xlabel('X (m)', fontsize=12)
ax1.set_ylabel('Y (m)', fontsize=12)
ax1.set_title('IMU Strapdown: Trajectory Drift', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_aspect('equal')

# Plot 2: Error over time
ax2 = axes[1]
ax2.plot(t, pos_error, 'r-', linewidth=2)
ax2.fill_between(t, 0, pos_error, alpha=0.3, color='red')
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('Position Error (m)', fontsize=12)
ax2.set_title('Position Error Growth (UNBOUNDED!)', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Key Insight: IMU drift makes standalone integration UNUSABLE!")
print("   Solutions: ZUPT, sensor fusion, or bounded DR methods (wheel odom, PDR)")


---

# Part 3: Pedestrian Dead Reckoning (PDR)

## 3.1 Theory (Eqs. 6.46-6.50)

PDR uses **step-and-heading** instead of double integration:

### Step Detection (Eq. 6.46)
Detect steps from accelerometer magnitude peaks:
$$a_{mag} = \sqrt{a_x^2 + a_y^2 + a_z^2}$$

### Step Length Estimation - Weinberg Model (Eq. 6.49)
$$L = k \cdot \sqrt[4]{a_{max} - a_{min}}$$

where $k$ is calibrated per user (typically $k \approx 0.4$)

### Position Update (Eq. 6.50)
$$\mathbf{p}_{k+1} = \mathbf{p}_k + L_k \begin{bmatrix} \cos(\psi_k) \\ \sin(\psi_k) \end{bmatrix}$$

**Advantage**: Bounded drift (errors don't grow quadratically)

**Challenge**: Heading accuracy is CRITICAL (1° error → ~1.7% position error per step)


In [ ]:
# Example 2: Pedestrian Dead Reckoning (PDR)
print("="*70)
print("Example 2: Pedestrian Dead Reckoning (Step-and-Heading)")
print("="*70)

# Simulate a corridor walk (L-shaped path)
np.random.seed(42)
step_length_true = 0.7  # meters
heading_noise_std = np.deg2rad(3.0)  # 3 degree heading noise
n_steps = 50

# True path: Walk 20m north, turn right, walk 15m east
waypoints = [
    (0, 0),      # Start
    (0, 20),     # Turn 1
    (15, 20),    # End
]

# Generate steps along the path
steps_north = 29  # 20m / 0.7m per step ≈ 29 steps
steps_east = 21   # 15m / 0.7m per step ≈ 21 steps

true_headings = []
true_positions = [(0, 0)]

# North leg
for _ in range(steps_north):
    true_headings.append(np.pi / 2)  # North = 90°
    last_pos = true_positions[-1]
    true_positions.append((last_pos[0], last_pos[1] + step_length_true))

# East leg
for _ in range(steps_east):
    true_headings.append(0.0)  # East = 0°
    last_pos = true_positions[-1]
    true_positions.append((last_pos[0] + step_length_true, last_pos[1]))

true_positions = np.array(true_positions)
true_headings = np.array(true_headings)

print(f"\n🚶 PDR Scenario:")
print(f"  Path: L-shaped (20m North → 15m East)")
print(f"  True step length: {step_length_true} m")
print(f"  Total steps: {len(true_headings)}")
print(f"  Heading noise: {np.rad2deg(heading_noise_std):.1f}°")


In [ ]:
# Run PDR with noisy heading
noisy_headings = true_headings + np.random.randn(len(true_headings)) * heading_noise_std

pdr_positions = [np.array([0.0, 0.0])]
for i, heading in enumerate(noisy_headings):
    new_pos = pdr_step_update(pdr_positions[-1], step_length_true, heading)
    pdr_positions.append(new_pos)
pdr_positions = np.array(pdr_positions)

# Compute errors
pdr_errors = np.linalg.norm(pdr_positions - true_positions, axis=1)
final_pdr_error = pdr_errors[-1]
total_distance_pdr = len(true_headings) * step_length_true

print(f"\n📊 PDR Results:")
print(f"  Total distance: {total_distance_pdr:.1f} m")
print(f"  Final error:    {final_pdr_error:.2f} m")
print(f"  Error %:        {100*final_pdr_error/total_distance_pdr:.1f}%")
print(f"  Max error:      {pdr_errors.max():.2f} m")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Trajectory
ax1 = axes[0]
ax1.plot(true_positions[:, 0], true_positions[:, 1], 'g-', linewidth=2, label='True Path')
ax1.plot(pdr_positions[:, 0], pdr_positions[:, 1], 'b--', linewidth=2, label='PDR (noisy heading)')
ax1.plot(0, 0, 'go', markersize=10, label='Start')
ax1.plot(true_positions[-1, 0], true_positions[-1, 1], 'g^', markersize=10, label='End (True)')
ax1.plot(pdr_positions[-1, 0], pdr_positions[-1, 1], 'b^', markersize=10, label='End (PDR)')
ax1.set_xlabel('East (m)', fontsize=12)
ax1.set_ylabel('North (m)', fontsize=12)
ax1.set_title('PDR Trajectory (L-shaped corridor)', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_aspect('equal')

# Plot 2: Error per step
ax2 = axes[1]
ax2.plot(range(len(pdr_errors)), pdr_errors, 'b-', linewidth=2)
ax2.axhline(y=final_pdr_error, color='r', linestyle='--', label=f'Final error: {final_pdr_error:.2f}m')
ax2.set_xlabel('Step Number', fontsize=12)
ax2.set_ylabel('Position Error (m)', fontsize=12)
ax2.set_title('PDR Error per Step', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 PDR drift is BOUNDED but heading accuracy is critical!")


---

# Part 4: Environmental Sensors

## 4.1 Magnetometer Heading (Eqs. 6.51-6.53)

Magnetometer provides **absolute heading** (compass direction):

$$\psi = \arctan\left(\frac{m_y}{m_x}\right) - \delta$$

where:
- $m_x, m_y$ = horizontal magnetic field components
- $\delta$ = magnetic declination (varies by location)

**Requires**: Tilt compensation using accelerometer

## 4.2 Barometric Altitude (Eq. 6.54)

Pressure-altitude relationship:

$$h = \frac{T_0}{L} \left[1 - \left(\frac{P}{P_0}\right)^{\frac{R \cdot L}{g \cdot M}}\right]$$

**Advantage**: Absolute altitude reference (unlike IMU z-integration)


In [ ]:
# Example 3: Environmental Sensors
print("="*70)
print("Example 3: Environmental Sensors (Magnetometer + Barometer)")
print("="*70)

# Magnetometer heading example
np.random.seed(42)
true_heading_deg = 45.0  # Northeast

# Simulated magnetometer readings (μT) with noise
mag_noise_std = 2.0  # μT
earth_field = 50.0  # μT horizontal component
mag_x = earth_field * np.cos(np.deg2rad(true_heading_deg)) + np.random.randn() * mag_noise_std
mag_y = earth_field * np.sin(np.deg2rad(true_heading_deg)) + np.random.randn() * mag_noise_std
mag_z = 0.0  # Assume level device

mag_vec = np.array([mag_x, mag_y, mag_z])
estimated_heading = mag_heading(mag_vec, roll=0.0, pitch=0.0, declination=0.0)

print(f"\n🧭 Magnetometer Heading:")
print(f"  True heading:      {true_heading_deg:.1f}°")
print(f"  Estimated heading: {np.rad2deg(estimated_heading):.1f}°")
print(f"  Error:             {np.rad2deg(estimated_heading) - true_heading_deg:.1f}°")

# Barometric altitude example
print(f"\n📊 Barometric Altitude:")
pressures = [101325, 100000, 95000, 90000]  # Pa
for p in pressures:
    alt = pressure_to_altitude(p)
    print(f"  {p/100:.0f} hPa → {alt:.1f} m")


---

# Summary

## Key Takeaways

### 1. Dead Reckoning Methods

| Method | Drift Type | Typical Error | Platform |
|--------|------------|---------------|----------|
| **IMU Strapdown** | Unbounded (∝ t²) | >1000% | - (needs fusion) |
| **Wheel Odometry** | Bounded (~1-5%) | ~1-5% of distance | Vehicles |
| **PDR** | Bounded (~2-5%) | ~2-5% of distance | Pedestrians |

### 2. Critical Insights

1. **IMU alone is UNUSABLE** - drift grows unbounded
2. **PDR is bounded** but heading is critical
3. **Environmental sensors** provide absolute references
4. **All DR methods need fusion** for long-term accuracy

### 3. Drift Correction Strategies

| Constraint | Effect | When |
|------------|--------|------|
| **ZUPT** | Reset velocity to zero | Stationary periods |
| **ZARU** | Reset angular rate to zero | No rotation |
| **NHC** | Constrain lateral velocity | Ground vehicles |
| **Absolute fixes** | Reset position | WiFi, UWB, GPS |

---

## Exercises

1. **Vary gyro bias**: How does 0.1°/s vs 5°/s bias affect IMU drift?
2. **PDR heading sources**: Compare gyro-only vs magnetometer heading
3. **Step length model**: Try different step length estimation methods

---

**Next Steps:** Chapter 7 (SLAM) for simultaneous localization and mapping, or Chapter 8 (Sensor Fusion) to combine DR with absolute positioning!
